# About: 元の運用環境に戻す

---

運用環境を移行前の元の運用環境に戻します。

## 概要

運用環境を移行前の環境に戻します。

![元の運用環境に戻す](images/moodle-192-01.png)

この Notebook は新しい運用環境に移行した後に、何らかの事情で運用環境を元の環境に戻す場合の手順です。

> このNotebookでは運用環境の切り替えのみを行い、移行前環境のリソースの削除は行いません。移行前の環境を削除するには「193-古い環境の削除.ipynb」を実行してください。

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないか、「010-パラメータ設定.ipynb」のNotebookを実行していないことが考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、VCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

元に戻す環境が存在していることを確認します。

In [ ]:
candidate = [
    x['project_tag']
    for x in gvars.get('previous_info_list', [])
    if ('project_tag' in x and
        x['project_tag'] not in gvars.get('delete_tags', []))
]
if len(candidate) == 0:
    raise RuntimeError("ERROR")

## パラメータの設定

元に戻す対象となる運用環境のタグの値を指定します。

指定できるタグ名の一覧を表示します。対象となるのは、以前使用していた運用環境であり、かつ削除していないものに限られます。

In [ ]:
candidate = [
    x['project_tag']
    for x in gvars.get('previous_info_list', [])
    if ('project_tag' in x and
        x['project_tag'] not in gvars.get('delete_tags', []))
]

for tag in candidate:
    print(tag)

上記のセルの実行結果として表示された値から、運用環境として戻す環境のタグ名を指定してください。

In [ ]:
# (例)
# target_tag = '001'
# target_tag = '20200101XXXXXX'

target_tag =

指定したタグ名が妥当かチェックします。次のセルを実行してエラーになる場合は指定したタグの値が正しくありません。値を指定し直してください。

In [ ]:
if target_tag not in candidate:
    raise RuntimeError("ERROR")

## メンテナンスモードへの切り替え

運用環境の切り替えを行う間に新たなデータが書き込まれないようにするために、メンテナンスモードに切り替えます。。

![メンテナンスモードへの切り替え](images/moodle-192-02.png)

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose exec -T moodle-{gvars["project_tag"]} \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

## 元の運用環境のコンテナを起動する

![コンテナの起動](images/moodle-192-03.png)

現在のコンテナ実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

次のセルを実行するとコンテナ設定を変更対象の運用環境（元の環境）の値に変更した `docker-compose.yml` をローカル環境に作成します。その後、更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` を編集することもできます。

In [ ]:
target_info = [
    x
    for x in gvars.get('previous_info_list', [])
    if ('project_tag' in x and
        x['project_tag'] == target_tag)
][0]
print(target_info)

%run scripts/edit_conf.py
update_docker_compose(ugroup_name, extra_vars=target_info)

更新した`docker-compose.yml`をVCノードに配置します。

In [ ]:
upload_docker_compose(ugroup_name)

`docker-compose.yml`の変更に対応するようコンテナを実行します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d'

変更後のコンテナ実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

## 元の運用環境に切り替える

リバースプロキシの設定を変更して、運用環境を切り替えます。

![元の運用環境に切り替える](images/moodle-192-04.png)

### リバースプロキシの設定

リバースプロキシの設定を変更します。

現在のリバースプロキシの設定ファイルを確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /opt/moodle/proxy/conf/moodle-proxy.conf'

次のセルを実行すると元の運用環境に切り替えるように記述したApache HTTP Serverの設定ファイル
`moodle-proxy.conf`をローカル環境に作成します。作成後に現在の設定ファイルとの差分を表示します。
また、ローカル環境に生成した `moodle-proxy.conf` を編集するためのリンクを最後に表示します。

In [ ]:
%run scripts/edit_conf.py
update_proxy_conf(ugroup_name, extra_vars=target_info)

ローカル環境の変更後の `moodle-proxy.conf` をVCノードに配置します。
その後、設定ファイルの記述内容を反映させるためにリバースプロキシコンテナを再起動します。

In [ ]:
apply_proxy_conf(ugroup_name)

再起動後のコンテナの状態を確認します。`proxy`コンテナの状態が `Up` になっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

### group_vars の更新

group_varsに記録している運用環境につけているタグの値を新しい環境の値に更新します。

In [ ]:
import copy

gvars = load_group_vars(ugroup_name)

# 切り替え後の運用環境に関する情報（元の運用環境）
next_info = [
    x for x in gvars['previous_info_list']
    if ('project_tag' in x and
        x['project_tag'] == target_tag)
][0]

# 切り替え前の運用環境に関する情報
previous_info = dict([(x, gvars[x]) for x in next_info.keys()])

# group_varsの更新
info_list = copy.copy(gvars['previous_info_list'])
info_list.remove(next_info)
info_list.insert(0, previous_info)
update_group_vars(
    ugroup_name,
    previous_info_list=info_list,
    **next_info)
gvars = load_group_vars(ugroup_name)

### 移行前の運用環境のコンテナを停止する

移行前の運用環境のコンテナを停止します。

コンテナを停止するまえの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

移行前の運用環境のコンテナの停止を行います。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d --remove-orphans'

実行しているコンテナの一覧を表示して、古い運用環境のコンテナが表示されないことを確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

### メンテナンスモードの解除

Moodleのメンテナンスモードを解除します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose exec -T moodle-{target_tag} \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

新しい運用環境のMoodle環境にアクセスして正しく動作していることを確認します。

次のセルを実行するとMoodleのアドレスが表示されます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/admin/index.php" target="_blank">{0}/admin/index.php</a>'.format(gvars['moodle_url']))